In [1]:
import cv2
import numpy as np
from itertools import count
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform, EssentialMatrixTransform
import matplotlib.pyplot as plt
from frame_helper import generate_frames, extractFeatures, bfmatcher

vid_path = 'test_freiburgrpy525.mp4'
kps_1 = None

for i, (p, c) in enumerate(generate_frames(vid_path)):

  if kps_1 is not None:
    kps_1, des_1 = kps_2, des_2
  else:
    kps_1, des_1 = extractFeatures(p)
  kps_2, des_2 = extractFeatures(c)

  p1, p2 = bfmatcher([kps_1, kps_2], [des_1, des_2])
  
  for p in p2:
    cv2.circle(c, (int(p[0]), int(p[1])), 1, (255,255,0))  
     
  for k1, k2 in zip(p1, p2):
    cv2.line(c, tuple(k1.astype(int)), tuple(k2.astype(int)), (0,200,200), 1)
  cv2.imshow('v', c)
  key = cv2.waitKey(1)
  if key == ord('q'):
    break

cv2.destroyAllWindows()


AttributeError: 'cv2.ORB' object has no attribute 'detectandcompute'